In [20]:
from konlpy.tag import *
import datetime
from IPython.display import display

import re
import numpy as np
import pandas as pd
from pprint import pprint
from konlpy.tag import Hannanum

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
df = pd.read_csv('tuned_df.csv')
df.tail()

,click,content,time,title,noun_tuned
8436,3190,이더리움 비트코인 비트코인과 함께 이더리움에 관심이 쏠리고 있다한 매체 보도에 따...,05-25,비트코인 이어 이더리움도 관심 스마트 콘트랙트 전자계약이 뭐길래,"['더', '리움', '비트코인', '비트코인', '더', '리움', '관심', '..."
8437,2706,비트코인 AFP뉴스1대표적인 전자화폐 비트코인의 랠리가 눈부시다 올 들어서만 그 ...,05-25,비트코인보다 눈부신 이더리움…올해 들어 2300 급등,"['비트코인', '뉴스', '대표', '전자화폐', '비트코인', '랠리', '그'..."
8438,3067,Photo pixabay비트코인Bitcoin이 화제다 비트코인은 2009년 나카모...,05-25,비트코인 이더리움 가상화폐 수요 급증 이유는,"['비트코인', '이', '화제', '비트코인', '나카', '모토', '사토시',..."
8439,2801,사진게티이미지뱅크 비트코인 가상화폐비트코인 이더리움 등 가상화폐의 가격이 계속해서 ...,05-25,몸값 오르는 가상화폐…비트코인·이더리움 뭐길래,"['사진', '티', '이미지', '뱅크', '비트코인', '가상', '화폐', '..."
8440,2341,비트코인이 2000달러를 돌파한지 닷새만에 또다시 최고치를 새로 썼다고 24일현지시...,05-25,비트코인 2500달러 또 사상최고치,"['비트코인', '돌파', '한지', '닷새', '최고', '보도', '비트코인',..."


In [3]:
data = df['noun_tuned'].tolist()

In [4]:
stop_words = list(pd.read_csv('kor_stop_words.csv').T.iloc[:1,:].values[0])
stop_words[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [12]:
data_lemmatized = remove_stopwords(data)

- LDA 모델에 들어가야 하는 두 가지 입력변수는 딕셔너리와(id2word) 코퍼스(corpus)이다.
- gensim 은 문서 내에 있는 단어별로 유니크한 아이디를 할당해준다.
- 아래의 각각의 엘리먼트 튜플당 의미하는 것은 [word_id,word_frequency] 이다.

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


만약 해당 id에 속한 단어를 보고싶으면

In [14]:
id2word[0]

'기회'

위의 표는 컴퓨터가 읽기 쉽게끔 만들어준 것이고, Counter 객체처럼 사람이 읽기 쉽게 만든 것은 아래와 같다.

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('기회', 1),
  ('넥시빗', 1),
  ('도전', 1),
  ('아이폰', 1),
  ('이벤트', 1),
  ('정말', 1),
  ('크게', 1),
  ('핸드폰', 1)]]

여태까지 해온 것이 LDA 모델 생성에 필요한 것들을 전부 한 것이다. 코퍼스와 딕셔너리를 생성한 것에 더해서, 우리는 몇 개의 토픽을 할당할 것인지에 대한 결정을 해주어야 한다.

- alpha , eta 는 토픽들의 떨어진 정도(sparsity)에 영향을 끼치는 하이퍼 파라미터이다. 도큐먼트에 따르면, 디폴트값은 1.0/num_topics prior 이다.
- chunksize 는 각각의 training chunk 에 사용될 문서의 갯수를 의미한다. 확실하지는 않지만, batch_size 와 유사한 의미를 갖는 것로 해석된다.

    - IN ADDITION : Text chunking, also referred to as shallow parsing, is a task that follows Part-Of-Speech Tagging and that adds more structure to the sentence. The result is a grouping of the words in “chunks”.


In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.4899471352874984

Coherence Score:  0.4121598797091758


pyLDAvis 만큼 jupyter notebook에서 LDA랄 잘 작동하면서 시각화하는 툴도 없다.

In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      31.462220        1       1 -0.152433  0.102535
2      30.945050        1       2 -0.173982 -0.149495
4      29.843639        1       3 -0.171779  0.158228
3       5.079869        1       4  0.147971 -0.275742
0       2.669223        1       5  0.350223  0.164473, topic_info=      Category          Freq   Term         Total  loglift  logprob
term                                                               
127    Default  24853.000000     화폐  24853.000000  30.0000  30.0000
215    Default  19660.000000     가상  19660.000000  29.0000  29.0000
54     Default  15138.000000     블록  15138.000000  28.0000  28.0000
96     Default  14871.000000     체인  14871.000000  27.0000  27.0000
102    Default   8643.000000     코인   8643.000000  26.0000  26.0000
55     Default  27457.000000   비트코인  27457.000000  25.0000  25.0000
379    Default   8984.000000     기술   8984.000000  24.0000  24.0000
105    Default   5932.000000     투자   5932.000000  23.0000  23.0000
352    Default   4147.000000     국내   4147.000000  22.0000  22.0000
611    Default   2431.000000     빗썸   2431.000000  21.0000  21.0000
14     Default   8974.000000    거래소   8974.000000  20.0000  20.0000
8      Default   6876.000000     가격   6876.000000  19.0000  19.0000
264    Default   2206.000000     사람   2206.000000  18.0000  18.0000
44     Default   8849.000000     리움   8849.000000  17.0000  17.0000
501    Default   3779.000000     채굴   3779.000000  16.0000  16.0000
698    Default   1987.000000     보안   1987.000000  15.0000  15.0000
431    Default   2154.000000     대표   2154.000000  14.0000  14.0000
13     Default  13602.000000     거래  13602.000000  13.0000  13.0000
104    Default   3179.000000     통화   3179.000000  12.0000  12.0000
29     Default   4087.000000     기업   4087.000000  11.0000  11.0000
2551   Default   1004.000000     공격   1004.000000  10.0000  10.0000
315    Default   2602.000000    플랫폼   2602.000000   9.0000   9.0000
232    Default   3112.000000     기반   3112.000000   8.0000   8.0000
25     Default   6124.000000     금융   6124.000000   7.0000   7.0000
46     Default   1720.000000     리플   1720.000000   6.0000   6.0000
853    Default   4218.000000    서비스   4218.000000   5.0000   5.0000
136    Default   1401.000000     상장   1401.000000   4.0000   4.0000
1232   Default   3247.000000     규제   3247.000000   3.0000   3.0000
1026   Default   2429.000000     경우   2429.000000   2.0000   2.0000
1284   Default   1051.000000    컴퓨터   1051.000000   1.0000   1.0000
...        ...           ...    ...           ...      ...      ...
1159    Topic5    141.109447     사칭    141.760681   3.6188  -5.2920
13226   Topic5    138.596773    크라이    139.238133   3.6188  -5.3100
2850    Topic5    136.065085    디도스    136.725997   3.6185  -5.3285
2182    Topic5    129.834850     피자    130.479813   3.6184  -5.3753
14357   Topic5    124.075326  보이스피싱    124.708837   3.6183  -5.4207
1537    Topic5    118.983649     터키    119.619847   3.6181  -5.4626
5979    Topic5    119.329966     광부    120.001431   3.6178  -5.4597
1901    Topic5    117.134956     추천    117.802007   3.6177  -5.4783
2658    Topic5    102.566964     환자    103.196900   3.6173  -5.6111
3835    Topic5    101.200442     우주    101.842725   3.6171  -5.6245
6564    Topic5    101.515632     몸값    102.173264   3.6169  -5.6214
2655    Topic5     92.055256     패턴     92.704496   3.6164  -5.7192
3363    Topic5     95.521980    휴대폰     96.226172   3.6160  -5.6822
12583   Topic5     73.255398    카메라     73.918265   3.6144  -5.9476
4210    Topic5    119.107180     악성    121.529095   3.6033  -5.4616
2551    Topic5    849.801301     공격   1004.089287   3.4565  -3.4966
893     Topic5    454.971840     해커    543.271715   3.4460  -4.1213
167     Topic5    309.332923    사이버    454.355369   3.2389  -4.5072
5101    Topic5    128.719578     탈취    145.052477   3.5039  -5.3839
735     Topic5    292.383991     주소    575.